This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key': API_KEY, 'limit': 1 }

request = requests.get(url, params = params)
json_data = request.json()
print(json_data)

{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-07-23', 'frequency': 'daily', 'data': [['2020-07-23', 88.4, 91.55, 88.15, 91.55, None, 146866.0, 13280553.85, None, None, None]], 'collapse': None, 'order': None}}


In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
dataset = json_data['dataset_data']
print(f"dataset is a {type(dataset)} with {dataset.keys()}.\n")
print(f"dataset contains data from {dataset['start_date']} to {dataset['end_date']}.\n")

print(f"columns names: {dataset['column_names']}\n")

rows = dataset['data']
print(f"rows is a {type(rows)}. total rows: {len(rows)}")
first_row = rows[0]
print(f"Each entry in a row is {type(first_row)}")
print(f"Data in the column name 'High' from the first row: {first_row[2]}")
print(f"Data in the column name 'Low' from the first row: {first_row[3]}")

dataset is a <class 'dict'> with dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order']).

dataset contains data from 2000-06-07 to 2020-07-23.

columns names: ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']

rows is a <class 'list'>. total rows: 1
Each entry in a row is <class 'list'>
Data in the column name 'High' from the first row: 91.55
Data in the column name 'Low' from the first row: 88.15


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
params = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31' }
request = requests.get(url, params = params)
json_data = request.json()
dataset = json_data['dataset_data']
rows = dataset['data']

In [7]:
opening_prices = [row[1] for row in rows if row[1] is not None]
max_opening_price = max(opening_prices)
min_opening_price = min(opening_prices)

print('''The highest opening price in 2017 was ${high_price:.2f} per share. 
The lowest opening price in 2017 was ${low_price:.2f} per share.''' \
      .format(high_price = round(max_opening_price, 2), low_price = round(min_opening_price, 2)))

The highest opening price in 2017 was $53.11 per share. 
The lowest opening price in 2017 was $34.00 per share.


In [8]:
deltas = [row[2] - row[3] for row in rows if row[2] is not None and row[3] is not None]
max_delta = max(deltas)

print("The largest daily price change in 2017 was {symbol}${price:.2f} per share."\
      .format(symbol = u'\u0394', price = round(max_delta, 2)))

The largest daily price change in 2017 was Δ$2.81 per share.


In [9]:
close_prices = [row[4] for row in rows if row[4] is not None]
deltas = [close_prices[i+1] - close_prices[i] for i in range(1, len(close_prices) - 1)]
max_delta = max(deltas)

print("The largest price change in 2017 between any two days was {symbol}${price:.2f} per share."\
      .format(symbol = u'\u0394', price = round(max_delta, 2)))

The largest price change in 2017 between any two days was Δ$2.56 per share.


In [10]:
trading_volumes = [row[6] for row in rows if row[6] is not None]
count = len(trading_volumes)

In [11]:
average_volume = sum(trading_volumes)/count if count > 0 else 0

print("The average daily trading volume in 2017 was {0} shares.".format(round(average_volume, 3)))

The average daily trading volume in 2017 was 89124.337 shares.


In [12]:
sorted_volumes = sorted(trading_volumes)
mid = int(count / 2)
median = (sorted_volumes[mid - 1] + sorted_volumes[mid]) / 2 if count % 2 == 0 else sorted_volumes[mid]

print("The median trading volume in 2017 was {0} shares.".format(round(median, 3)))

The median trading volume in 2017 was 76286.0 shares.
